#Campaign Analytics<br/>

1. **Usecase               :** Performing Campaign analytics on static campaign data coming from snowflake in azure container.<br/>
2. **Notebook Summary      :** This notebook is a part of campaign analytics application which perform `campaign analytics using various pyspark capability`.<br/>
3. **Notebook Description  :** Performing Campaign Analytics on Azure Container Files.


###Feature List
1. Data Profiling
2. Total Revenue Target & Profit 
3. Campaign Run by Per Week 
4. Total Profit by Country Per Week
5. Top Loss-Making Campaign 

The bronze data received for processing is already curated. So, we will derive gold tables from bronze tables.

### Import Libraries

In [0]:
import dlt
from pyspark.sql.functions import sum as _sum
from pyspark.sql.functions import mean as _mean
from pyspark.sql.functions import max as _max
from pyspark.sql.functions import min as _min
import pyspark.sql.functions as func
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType

###Define the Schema for the input file

In [0]:
campaignSchema = StructType([    
    StructField("Region",StringType(),True),
    StructField("Country",StringType(),True),
    StructField("ProductCategory",StringType(),True),
    StructField("Campaign_ID",IntegerType(),True),    
    StructField("Campaign_Name",StringType(),True),
    StructField("Qualification",StringType(),True),
    StructField("Qualification_Number",StringType(),True),
    StructField("Response_Status",StringType(),True),
    StructField("Responses",FloatType(),True),
    StructField("Cost",FloatType(),True),
    StructField("Revenue",FloatType(),True),
    StructField("ROI",FloatType(),True),
    StructField("Lead_Generation",StringType(),True),
    StructField("Revenue_Target",FloatType(),True),
    StructField("Campaign_Tactic",StringType(),True),
    StructField("Customer_Segment",StringType(),True),
    StructField("Status",StringType(),True),
    StructField("Profit",FloatType(),True),
    StructField("Marketing_Cost",FloatType(),True),
    StructField("CampaignID",IntegerType(),True),
    StructField("CampDate",DateType(),True), 
    StructField("SORTED_ID",IntegerType(),True)])
    

### Load the Campaign Dataset from Data Lake

In [0]:
# Bronze Table Setup
@dlt.table(comment="Raw data")
def bronze_campaign_data():
#   return (spark.table("campaign.campaign_source"))
  return (spark.read.format("csv").option("header",True).schema(campaignSchema).load("/mnt/data-source/Campaign Data/campaign-data2.csv"))

### Total Revenue Target & Profit

In [0]:
# Gold Table Setup
@dlt.table(comment="Raw data")
def gold_country_wise_revenue():
    
    df = dlt.read("bronze_campaign_data").groupBy("Country","Region").agg(_sum("Revenue").alias("Total_Revenue"), _sum("Revenue_Target").alias("Total_Revenue_Target"),_sum("Profit").alias("Total_Profit"),_max("Cost").alias("Max_Cost"),_min("Cost").alias("Min_Cost"))
    df = df.withColumn("Total_Revenue", func.round(df["Total_Revenue"],2)).withColumn("Total_Revenue_Target", func.round(df["Total_Revenue_Target"], 2)).withColumn("Total_Profit", func.round(df["Total_Profit"], 2))
    return df
    

### Campaign Run by Per Week

In [0]:
# Gold Table Setup
@dlt.table(comment="Raw data")
def gold_campaign_per_week():
    return dlt.read("bronze_campaign_data") \
    .groupBy(
      "Campaign_Name",
      window("CampDate", "1 week")) \
    .count().orderBy(col("count").desc())

### Total Profit by Country Per Week

In [0]:
# Gold Table Setup
@dlt.table(comment="Raw data")
def gold_Total_Profit_by_Country_Per_Week():
    return dlt.read("bronze_campaign_data").select("Region","Country", "Cost", "Profit","CampDate") \
                     .groupBy(window(col("CampDate"), "7 days"), col("Country")) \
                     .agg(sum("Profit").alias('Total_Profit'),) \
                            .orderBy(col("window.start"))

### Campaign Analysis

In [0]:
# Gold Table Setup
@dlt.table(comment="Raw data")
def gold_Top_Loss_Making_Campaign():
    loss = dlt.read("bronze_campaign_data").select("Campaign_Name","Profit").filter(F.col("Profit") < 0)
    loss = loss.withColumn("Loss_Count", F.when((F.col('Profit') < 0 ) , F.lit(1)).otherwise(F.lit(0)))
#     loss = loss.groupBy('Campaign_Name').sum('Loss_Count')
    return loss